In [2]:
import pickle
import pandas as pd
import numpy as np
from pyspark.sql import DataFrame
from aeon.classification.distance_based import ProximityTree, ProximityForest
import logging

from pyspark.sql import SparkSession
import os
from pyspark.sql import SparkSession
from data_ingestion import DataIngestion
from preprocessing import Preprocessor
from prediction_manager import PredictionManager
from local_model_manager import LocalModelManager
from evaluation import Evaluator
from utilities import show_compact
import time
import json

In [3]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("ProximityTree") \
    .getOrCreate()

sc = spark.sparkContext

In [4]:
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, "..", ".."))

config_DataIngestion = {
            # "data_path": project_root + "/ECG5000/*.tsv" # IF WE DECIDE TO USE .tsv FILES
            "data_path": project_root + "/fulldataset_ECG5000.csv", # IF WE DECIDE TO USE .csv FILES
            "data_percentage": 0.5  #*100% Percentage of data to load for SW development
            }   

ingestion = DataIngestion(spark,config_DataIngestion)
df = ingestion.load_data()
print(df.show(5))
df.printSchema()

Data Path is c:\Users\benat\OneDrive\0. MSc MLiS\0. GitHub Repositories\BigDataProject_repo/fulldataset_ECG5000.csv and loading 50.0% of data ++++++++++++++++++++++

Data size is :2493

+-----+-----------+-----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+-----------+------------+------------+------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+-----------+-----------+-----------+----------+-----------+-----------+-----------+----------+----------+----------+----------+----------

In [5]:
config_preproc = {}    
preprocessor = Preprocessor(config_preproc)

preprocessed_df = preprocessor.run_preprocessing(df)
print(preprocessed_df.show(5))

+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------

In [6]:
df.count()

2493

In [7]:
num_partitions = df.rdd.getNumPartitions() # Number of partitions in the DataFrame
num_partitions

2

In [8]:
for partition in range(df.rdd.getNumPartitions()):
    print(f"Number of rows in partition {partition+1}: {len(df.rdd.glom().collect()[partition])}")
    

Number of rows in partition 1: 1318
Number of rows in partition 2: 1175


In [9]:
n_splitters = 5

fraction = n_splitters / df.count()  # Fraction of data to sample for each splitter
fraction

0.0020056157240272766

In [10]:
splitters = df.rdd.glom().sample(fraction=0.25, withReplacement=False).collect()
splitters

[]

In [14]:
from random import sample

def sample_from_partition(index, iterator):
    if index==0:
        partition_data = list(iterator)
        # Randomly sample 5 values (or fewer if the partition has less than 5 rows)
        return sample(partition_data, min(5, len(partition_data)))
    else:
        return []
    
# def sample_from_partition(index, iterator):
#     for index in range(num_partitions):
#         partition_data = list(iterator)
#         # Randomly sample 5 values (or fewer if the partition has less than 5 rows)
#         return sample(partition_data, min(5, len(partition_data)))
    
splitters = df.rdd.mapPartitionsWithIndex(sample_from_partition) # stays within the partition
collected_splitters = splitters.collect() # collects to the driver
print("Randomly sampled data from partition 1:", collected_splitters)

len(collected_splitters)  # Number of sampled rows

Randomly sampled data from partition 1: [Row(label=1, _c1=-1.998602, _c2=-3.7705516, _c3=-4.2670909, _c4=-4.2561327, _c5=-3.5152879, _c6=-2.5545403, _c7=-1.6996388, _c8=-1.5663663, _c9=-1.0388154, _c10=-0.42548335, _c11=-0.26247407, _c12=-0.19736084, _c13=-0.34049721, _c14=-0.32377442, _c15=-0.33551317, _c16=-0.25458288, _c17=-0.26433363, _c18=-0.33874005, _c19=-0.28464858, _c20=-0.368613, _c21=-0.3942132, _c22=-0.43972604, _c23=-0.36899342, _c24=-0.3767418, _c25=-0.42302235, _c26=-0.52946416, _c27=-0.58924777, _c28=-0.54400752, _c29=-0.51812932, _c30=-0.67854437, _c31=-0.71757742, _c32=-0.74004149, _c33=-0.76923281, _c34=-0.78521282, _c35=-0.82168545, _c36=-0.76925636, _c37=-0.77268609, _c38=-0.78158572, _c39=-0.81840365, _c40=-0.76908036, _c41=-0.6464461, _c42=-0.56546088, _c43=-0.53035286, _c44=-0.461239, _c45=-0.35378046, _c46=-0.34535436, _c47=-0.24656732, _c48=-0.13720012, _c49=-0.12506497, _c50=-0.14630101, _c51=-0.097486799, _c52=-0.050911484, _c53=-0.065833192, _c54=-0.0965732

5

In [12]:
from dtaidistance import dtw

ts1 = splitters[0]
ts2 = splitters[0]

dtw.distance(ts1, ts2)  # Calculate the DTW distance between the two time series

TypeError: 'PipelinedRDD' object is not subscriptable

In [13]:
def calculate_dtw(ts1, ts2):
    return dtw.distance(ts1, ts2)

splitters.mapPartitions(lambda x: [calculate_dtw(x[0], x[1])]).collect()  # Calculate DTW distance for each pair of time series in the partition

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 19.0 failed 1 times, most recent failure: Lost task 1.0 in stage 19.0 (TID 24) (BenAtkinson-Dell-Inspiron3505.Home executor driver): java.net.SocketException: Connection reset by peer
	at java.base/sun.nio.ch.NioSocketImpl.implWrite(NioSocketImpl.java:425)
	at java.base/sun.nio.ch.NioSocketImpl.write(NioSocketImpl.java:445)
	at java.base/sun.nio.ch.NioSocketImpl$2.write(NioSocketImpl.java:831)
	at java.base/java.net.Socket$SocketOutputStream.write(Socket.java:1035)
	at java.base/java.io.BufferedOutputStream.flushBuffer(BufferedOutputStream.java:81)
	at java.base/java.io.BufferedOutputStream.write(BufferedOutputStream.java:122)
	at java.base/java.io.DataOutputStream.write(DataOutputStream.java:112)
	at java.base/java.io.FilterOutputStream.write(FilterOutputStream.java:108)
	at org.apache.spar

In [15]:
type(splitters)  # Check the type of the splitters variable

pyspark.rdd.PipelinedRDD

In [ ]:
from random import sample
from pyspark.sql import Row

def annotate_with_splitters(index, iterator):
    """
    Annotate each row in the partition with the selected splitters.
    """
    partition_data = list(iterator)
    if len(partition_data) == 0:
        return []
    
    # Randomly sample 5 splitters from the partition
    splitters = sample(partition_data, min(5, len(partition_data)))
    
    # Annotate each row with the splitters by creating a new Row or dictionary
    annotated_data = []
    for row in partition_data:
        # Convert the Row to a dictionary, add the splitters, and convert back to a Row
        row_dict = row.asDict()  # Convert Row to a dictionary
        row_dict['splitters'] = splitters  # Add the splitters field
        annotated_data.append(Row(**row_dict))  # Convert back to a Row
    
    return annotated_data

# Apply the function to annotate the data in each partition
annotated_rdd = df.rdd.mapPartitionsWithIndex(annotate_with_splitters)

# Inspect the annotated data
annotated_data = annotated_rdd.collect()
print(annotated_data[:5])  # Print the first 5 rows
len(annotated_data)  # Check the number of rows in the annotated data

In [ ]:
df.rdd